In [2]:
%matplotlib inline
%load_ext rpy2.ipython

import requests
import pandas as pd
import numpy as np
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from os import listdir

In [3]:
df_rhist = pd.DataFrame.from_csv('rating_histories.csv')

In [5]:
# for each user, we want to normalize times to when they first started


,contestName,handle,newRating,oldRating,rank,ratingUpdateTimeSeconds
contestId,,,,,,
659,Codeforces Round #346 (Div. 2),Hedgehogy,292,372,3594,1459362900
669,"Codeforces Round #348 (VK Cup 2016 Round 2, Di...",Hedgehogy,222,292,2073,1461522900
670,Codeforces Round #350 (Div. 2),Hedgehogy,152,222,3230,1462473300
673,"Codeforces Round #351 (VK Cup 2016 Round 3, Di...",Hedgehogy,71,152,2328,1462644300
672,Codeforces Round #352 (Div. 2),Hedgehogy,14,71,3200,1462991700
